In [9]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


In [10]:
import utils

In [11]:
pd.set_option("display.max_columns", 100)

In [12]:
analysis_date = dt.date(2022,5,25)

In [13]:
analysis_date.isoformat()

'2022-05-25'

In [14]:
itp_id = 300

In [15]:
#route = 3479

In [16]:
sched = (tbl.views.gtfs_schedule_fact_daily_trips()
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
      >> collect()
)

In [17]:
sched

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
0,4100255323842349957,-7861484763504641269,883388,3496,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,21,21,61200,65640,1.233333
1,4100255323842349957,6196399810198140026,883374,3496,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,21,21,43200,46440,0.900000
2,4100255323842349957,-8909874686781619988,883385,3496,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,21,21,57600,62040,1.233333
3,4100255323842349957,-3158943638548239834,883419,3496,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,23,23,37500,41040,0.983333
4,4100255323842349957,-8190289866634217095,883546,3498,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,29,31,55200,57060,0.516667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511,4100255323842349957,-6344409580421376528,881848,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,53,53,36120,39771,1.014167
1512,4100255323842349957,-2809777294863142351,881873,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,53,53,72240,75361,0.866944
1513,4100255323842349957,-1584924129425044220,881907,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,48240,52080,1.066667
1514,4100255323842349957,-1301012917580333717,881904,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,49920,53760,1.066667


In [18]:
sched.service_id.nunique()

6

In [19]:
sched.service_id.value_counts()

10       1409
40510      52
77710      52
92610       1
12010       1
96910       1
Name: service_id, dtype: int64

In [20]:
(sched
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
 >> filter(_.calitp_extracted_at <= analysis_date, _.calitp_deleted_at >= analysis_date)
 >> summarize(unique= _.trip_id.nunique())
      
)

,unique
0,1516


In [21]:
(sched
 >>filter(_.calitp_itp_id==itp_id)
 >>filter(_.service_date == analysis_date)
 >>filter(_.is_in_service == True)
 >> filter(_.calitp_extracted_at <= analysis_date, _.calitp_deleted_at >= analysis_date)
 >> group_by(_.route_id) 
 >> summarize(n=_.trip_id.nunique()) >> arrange(-_.n))

,route_id,n
0,3479,180
4,3485,153
8,3489,153
2,3481,145
9,3490,107
18,3501,104
12,3493,88
1,3480,82
15,3496,76
5,3486,71


In [22]:
rt = query_sql(f'''
SELECT * FROM `cal-itp-data-infra-staging.natalie_views.gtfs_rt_distinct_trips`
WHERE date = '{analysis_date}'
''')

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [23]:
rt

,calitp_itp_id,calitp_url_number,date,trip_id,trip_route_id,vehicle_id
0,310,1,2022-05-25,None,None,1
1,200,0,2022-05-25,None,None,1
2,269,0,2022-05-25,None,None,1
3,148,0,2022-05-25,None,None,1
4,243,0,2022-05-25,None,None,1
...,...,...,...,...,...,...
104058,372,0,2022-05-25,6fc0b846-08f1-47e3-a821-33e5ada49d3e,1c17bb5c-1a20-4427-a01f-d5158c5cc5e2,ffa92186-27ae-47e6-9275-ab79fc82919a
104059,372,0,2022-05-25,a117e40e-d6d4-47b5-9f75-9bd67c2679c1,4bc2bfa4-be32-47c1-9a57-11a815f7429b,ffa92186-27ae-47e6-9275-ab79fc82919a
104060,372,0,2022-05-25,a5542b36-3a3b-40aa-aac5-e97e47300e6f,4bc2bfa4-be32-47c1-9a57-11a815f7429b,ffa92186-27ae-47e6-9275-ab79fc82919a
104061,372,0,2022-05-25,db2e11fe-2cdf-4a6e-bfe8-0d6f978dd526,1c17bb5c-1a20-4427-a01f-d5158c5cc5e2,ffa92186-27ae-47e6-9275-ab79fc82919a


In [24]:
sched>>distinct(_.trip_id)

,trip_id
0,883388
1,883374
2,883385
3,883419
4,883546
...,...
1511,881848
1512,881873
1513,881907
1514,881904


In [25]:
sched.sample()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id,calitp_url_number,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours
590,4100255323842349957,7774959311094125060,881894,3485,300,0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,54,54,59400,63360,1.1


In [26]:
len(sched)

1516

In [27]:
test_join = sched>>inner_join(_, rt, on = 'trip_id')

In [28]:
len(test_join)

1453

In [29]:
sched_v_rt = (pd.merge(sched, rt, how='outer', on='trip_id', indicator='have_rt'))

In [30]:
sched_v_rt.have_rt.value_counts()

right_only    102610
both            1453
left_only        168
Name: have_rt, dtype: int64

In [31]:
('{:.1%}'.format(1-(len(sched_v_rt>>filter(_.have_rt=='left_only')))/(len(sched_v_rt))))

'99.8%'

In [32]:
('{:.1%}'.format((len(sched_v_rt>>filter(_.have_rt=='left_only')))/(len(sched_v_rt))))

'0.2%'

In [33]:
('{:.1%}'.format((len(sched_v_rt>>filter(_.have_rt=='both')))/(len(sched_v_rt))))

'1.4%'

In [34]:
pct_not_ran = (
    (len(sched_v_rt>>filter(_.have_rt=='left_only')))
    /
    (len(sched_v_rt))
)
pct_not_ran = ('{:.1%}'.format(pct_not_ran))
pct_not_ran

'0.2%'

In [35]:
pct_ran = ((len(sched_v_rt>>filter(_.have_rt=='both')))/(len(sched_v_rt)))
pct_ran = ('{:.1%}'.format(pct_ran))
pct_ran

'1.4%'

In [36]:
display(Markdown(f"On {analysis_date.strftime('%B %d')}, **{pct_ran}** "
                "of scheduled trips produced GTFS RT data"))

On May 25, **1.4%** of scheduled trips produced GTFS RT data

In [37]:
# checking the length of the string to see if there are any incomplete/null `trip_ids`

In [38]:
rt>>filter(_.trip_id.isnull())

,calitp_itp_id,calitp_url_number,date,trip_id,trip_route_id,vehicle_id
0,310,1,2022-05-25,None,None,1
1,200,0,2022-05-25,None,None,1
2,269,0,2022-05-25,None,None,1
3,148,0,2022-05-25,None,None,1
4,243,0,2022-05-25,None,None,1
...,...,...,...,...,...,...
102904,182,1,2022-05-25,None,None,545-546-547-548-559-560
102952,182,1,2022-05-25,None,None,551-552-567-568-589-590
102994,182,1,2022-05-25,None,None,569-570-601-602-603-604
103025,182,1,2022-05-25,None,None,571-572-587-588-595-596


In [39]:
rt['str_len'] = rt.trip_id.str.len()

In [40]:
rt>>arrange(-_.str_len)

,calitp_itp_id,calitp_url_number,date,trip_id,trip_route_id,vehicle_id,str_len
103129,14,0,2022-05-25,25d5bfdc-5ec0-41f7-a46a-7d35ec852c6f:10,None,2066929f-ccdf-4dc0-ab3d-637558b56578,39.0
103130,14,0,2022-05-25,25d5bfdc-5ec0-41f7-a46a-7d35ec852c6f:11,None,2066929f-ccdf-4dc0-ab3d-637558b56578,39.0
103131,14,0,2022-05-25,25d5bfdc-5ec0-41f7-a46a-7d35ec852c6f:12,None,2066929f-ccdf-4dc0-ab3d-637558b56578,39.0
103132,14,0,2022-05-25,25d5bfdc-5ec0-41f7-a46a-7d35ec852c6f:13,None,2066929f-ccdf-4dc0-ab3d-637558b56578,39.0
103133,14,0,2022-05-25,25d5bfdc-5ec0-41f7-a46a-7d35ec852c6f:14,None,2066929f-ccdf-4dc0-ab3d-637558b56578,39.0
...,...,...,...,...,...,...,...
102904,182,1,2022-05-25,None,None,545-546-547-548-559-560,NaN
102952,182,1,2022-05-25,None,None,551-552-567-568-589-590,NaN
102994,182,1,2022-05-25,None,None,569-570-601-602-603-604,NaN
103025,182,1,2022-05-25,None,None,571-572-587-588-595-596,NaN


In [41]:
## Putting into a function

In [42]:
def find_ran_trips(itp_id, analysis_date):
    gtfs_daily = (
        tbl.views.gtfs_schedule_fact_daily_trips()
        >> filter(_.calitp_itp_id == itp_id)
        >> filter(_.service_date == analysis_date)
        >> filter(_.is_in_service == True)
        >> collect()
    )
    rt = query_sql(
        f"""
        SELECT * FROM `cal-itp-data-infra-staging.natalie_views.gtfs_rt_distinct_trips`
        WHERE date = '{analysis_date}'
        """
    )
    
    rt['str_len'] = rt.trip_id.str.len()
    
    #joinging 
    join = (pd.merge(gtfs_daily, rt, how='outer', on='trip_id', indicator='have_rt'))
    
    return join

def get_pct_ran(join, date):
    ## eventually generate a dataframe with the columns (dates and pct_ran)
    day_pct_ran = {}
    day_pct_ran['date'] = date
    day_pct_ran['pct_trips_ran'] = ((len(join>>filter(_.have_rt=='both')))/(len(join)))
    
    pct_ran = pd.DataFrame([day_pct_ran])

    return pct_ran


In [43]:
join_day1 = find_ran_trips(itp_id, analysis_date)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [44]:
join_day1.head()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date,trip_route_id,vehicle_id,str_len,have_rt
0,4.100255e+18,-7.861485e+18,883388,3496,300.0,0.0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,21.0,21.0,61200.0,65640.0,1.233333,300.0,0.0,2022-05-25,None,1701,6.0,both
1,4.100255e+18,6.196400e+18,883374,3496,300.0,0.0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,21.0,21.0,43200.0,46440.0,0.900000,300.0,0.0,2022-05-25,None,5312,6.0,both
2,4.100255e+18,-8.909875e+18,883385,3496,300.0,0.0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,21.0,21.0,57600.0,62040.0,1.233333,300.0,0.0,2022-05-25,None,1701,6.0,both
3,4.100255e+18,-8.909875e+18,883385,3496,300.0,0.0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,21.0,21.0,57600.0,62040.0,1.233333,300.0,0.0,2022-05-25,None,5312,6.0,both
4,4.100255e+18,-3.158944e+18,883419,3496,300.0,0.0,10,2022-05-25,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,23.0,23.0,37500.0,41040.0,0.983333,300.0,0.0,2022-05-25,None,5313,6.0,both


In [45]:
pct_day1 = get_pct_ran(join_day1, analysis_date)
pct_day1

,date,pct_trips_ran
0,2022-05-25,0.01394


In [46]:
# add another date into the mix

In [47]:
analysis_date2 = dt.date(2022,5,26)

In [48]:
join_day2 = find_ran_trips(itp_id, analysis_date2)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [49]:
join_day2.head()

,feed_key,trip_key,trip_id,route_id,calitp_itp_id_x,calitp_url_number_x,service_id,service_date,service_indicator,service_start_date,service_end_date,service_inclusion,service_exclusion,is_in_service,calitp_extracted_at,calitp_deleted_at,n_stops,n_stop_times,trip_first_departure_ts,trip_last_arrival_ts,service_hours,calitp_itp_id_y,calitp_url_number_y,date,trip_route_id,vehicle_id,str_len,have_rt
0,4.100255e+18,6.242274e+18,882298,3487,300.0,0.0,12010,2022-05-26,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2022-07-18,12.0,12.0,51240.0,53340.0,0.583333,300.0,0.0,2022-05-26,None,1336,6.0,both
1,4.100255e+18,-7.061414e+18,882562,3489,300.0,0.0,10,2022-05-26,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,14.0,14.0,30780.0,32400.0,0.450000,300.0,0.0,2022-05-26,None,1827,6.0,both
2,4.100255e+18,-8.321162e+18,882492,3489,300.0,0.0,10,2022-05-26,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,14.0,14.0,23460.0,24720.0,0.350000,300.0,0.0,2022-05-26,None,2113,6.0,both
3,4.100255e+18,5.427044e+18,883627,3501,300.0,0.0,40510,2022-05-26,0,2022-03-27,2022-08-13,True,None,True,2022-03-29,2022-07-18,14.0,14.0,36660.0,37620.0,0.266667,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,4.100255e+18,3.324965e+18,882634,3489,300.0,0.0,10,2022-05-26,1,2022-03-27,2022-08-13,None,None,True,2022-03-29,2022-07-18,17.0,17.0,73800.0,75120.0,0.366667,300.0,0.0,2022-05-26,None,1822,6.0,both


In [50]:
join_day2.have_rt.value_counts()

right_only    103056
both            1498
left_only        122
Name: have_rt, dtype: int64

In [51]:
pct_day2 = get_pct_ran(join_day2, analysis_date2)
pct_day2

,date,pct_trips_ran
0,2022-05-26,0.014311


In [52]:
pdList = [pct_day1, pct_day2]  # List of your dataframes
new_df = pd.concat(pdList)

In [53]:
new_df

,date,pct_trips_ran
0,2022-05-25,0.013940
0,2022-05-26,0.014311


In [54]:
# #another option for more dates... this works
# pdList2 = []
# pdList2.extend(value for name, value in locals().items() if name.startswith('pct_da'))
# new_df2 = pd.concat(pdList2)
# new_df2

In [55]:
date_list =['2022-05-25', '2022-05-26']

In [56]:
def get_pct_ran_df(date_list):
    pcts = []
    for date in date_list: 
        sched_rt_df = (find_ran_trips(itp_id, date))
        pct_ran = (get_pct_ran(sched_rt_df, date))
        pcts.append(pct_ran)                                                    
        #code help from: https://stackoverflow.com/questions/28669482/appending-pandas-dataframes-generated-in-a-for-loop
    pcts = pd.concat(pcts)
    return pd.DataFrame(pcts)


In [57]:
final = get_pct_ran_df(date_list)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [58]:
final

,date,pct_trips_ran
0,2022-05-25,0.013940
0,2022-05-26,0.014311


In [59]:
import altair as alt

In [60]:
chart = (
    alt.Chart(final)
    .mark_line()
    .encode(x="date", y="pct_trips_ran")
)

yrule = (
    alt.Chart().mark_rule(strokeDash=[12, 6], size=2).encode(y=alt.datum(1))
)


chart + yrule

alt.LayerChart(...)

In [ ]:
# def get_pct_ran_list():
    
#     pdList2 = []
#     pdList2.extend(value for name, value in locals().items() if name.startswith('pct_da'))
#     new_df = pd.concat(pdList2)
#     return new_df

In [ ]:
## other queries

In [ ]:
# (tbl.views.gtfs_rt_fact_daily_feeds
#  >>filter(_.calitp_itp_id==itp_id)
#  >>filter(_.date==analysis_date)
# )

In [ ]:
# trips =(tbl.views.gtfs_schedule_dim_trips()
#  >>filter(_.calitp_itp_id==itp_id) >> collect())


In [3]:
#trips

In [4]:
#pd.merge(df, trips, on="route_id")

In [5]:
# (tbl.views.gtfs_schedule_fact_daily_trips()
#  >>filter(_.calitp_itp_id==itp_id)
#  >>filter(_.service_date == analysis_date)
# )

In [6]:
# (
#     tbl.views.gtfs_rt_fact_daily_feeds()
#     >> filter(_.calitp_itp_id == itp_id)
#     >> arrange(_.date ==analysis_date)
# )

In [7]:
# airtable_organizations = (
#     tbl.airtable.california_transit_organizations()
#     >> select(_.itp_id, _.name, _.caltrans_district,
#               _.website, _.ntp_id, _.drmt_organization_name)
#     >> filter(_.itp_id == itp_id)
#     >> collect()
# )

In [8]:
# airtable_organizations